<a href="https://colab.research.google.com/github/KushagraGoel2024/Quote-analysis-author-and-sentiment/blob/main/Sentiment_analysis_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow scikit-learn pandas numpy pickle5
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM, Bidirectional, SimpleRNN
import pickle5 as pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255323 sha256=ad32dbe80090544d19a23038e4cf955adb253355895b1124113ab407af96a527
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/quotes.csv')
df = df[['quote', 'tags']]
df1=df
print(df1)
df1 = df1.sample(frac=1).reset_index(drop=True)

                                                  quote           tags
0        “Be yourself; everyone else is already taken.”  inspirational
1     “I'm selfish, impatient and a little insecure....   deep meaning
2     “Two things are infinite: the universe and hum...          humor
3                      “So many books, so little time.”   deep meaning
4     “A room without books is like a body without a...   deep meaning
...                                                 ...            ...
2500  “Morality is simply the attitude we adopt towa...          humor
2501  “Don't aim at success. The more you aim at it ...   deep meaning
2502  “In life, finding a voice is speaking and livi...  inspirational
2503  “Winter is the time for comfort, for good food...          humor
2504                      “Silence is so freaking loud”          humor

[2505 rows x 2 columns]


In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df1['quote'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df1['quote'])
padded_sequences = pad_sequences(sequences, maxlen=500, truncating='post')
tagslabels = pd.get_dummies(df1['tags']).values
print(pd.get_dummies(df1['tags']))

      deep meaning  humor  inspirational  love
0                1      0              0     0
1                1      0              0     0
2                0      0              1     0
3                0      0              1     0
4                0      0              1     0
...            ...    ...            ...   ...
2500             0      0              1     0
2501             0      1              0     0
2502             0      0              1     0
2503             0      0              0     1
2504             0      0              1     0

[2505 rows x 4 columns]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, tagslabels, test_size=0.3)

In [ ]:
model = Sequential()
model.add(Embedding(5000, 100, input_length=500))
model.add(SimpleRNN(200,return_sequences=True))
model.add(SimpleRNN(200,return_sequences=True))
model.add(Bidirectional(LSTM(200)))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          500000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 500, 200)          60200     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 500, 200)          80200     
                                                                 
 bidirectional (Bidirectiona  (None, 400)              641600    
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 1604      
                                                                 
Total params: 1,283,604
Trainable params: 1,283,604
Non-trainable params: 0
______________________________________________

In [ ]:
model.fit(x_train, y_train, epochs=50 , batch_size=32, validation_data=(x_test, y_test))

Epoch 1/50
55/55 [==============================] - 282s 5s/step - loss: 1.1591 - accuracy: 0.5003 - val_loss: 1.1815 - val_accuracy: 0.5000
Epoch 2/50
55/55 [==============================] - 268s 5s/step - loss: 1.1247 - accuracy: 0.5231 - val_loss: 1.1892 - val_accuracy: 0.5000
Epoch 3/50
55/55 [==============================] - 266s 5s/step - loss: 1.1261 - accuracy: 0.5231 - val_loss: 1.1807 - val_accuracy: 0.5000
Epoch 4/50
55/55 [==============================] - 274s 5s/step - loss: 1.1306 - accuracy: 0.5106 - val_loss: 1.1771 - val_accuracy: 0.5000
Epoch 5/50
55/55 [==============================] - 270s 5s/step - loss: 1.1128 - accuracy: 0.5237 - val_loss: 1.1755 - val_accuracy: 0.5000
Epoch 6/50
55/55 [==============================] - 271s 5s/step - loss: 1.1163 - accuracy: 0.5237 - val_loss: 1.1927 - val_accuracy: 0.5000
Epoch 7/50
55/55 [==============================] - 273s 5s/step - loss: 1.1111 - accuracy: 0.5237 - val_loss: 1.1804 - val_accuracy: 0.5000
Epoch 8/50
55

In [ ]:
model.save('/content/gdrive/MyDrive/sentiment_analysis_model.h5')

In [ ]:
import keras
model = keras.models.load_model('/content/gdrive/MyDrive/sentiment_analysis_model.h5')


In [ ]:
def predict_sentiment(text):
    text_sequence = tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=500)
    return model.predict(text_sequence)

In [ ]:
for n in range(200,2505):
  text_input =(df.iloc[n]['quote'])
  predicted_entiment = predict_sentiment(text_input)
  for i in predicted_entiment:
    d=max(i)
    for n in range(0,4):
      if i[n]==d:
        if n==0:
          print('deep meaning')
        if n==1:
          print('humor')
        if n==2:
          print('inspirational')
        if n==3:
          print('love')

1/1 [==============================] - 0s 181ms/step
deep meaning
1/1 [==============================] - 0s 188ms/step
deep meaning
1/1 [==============================] - 0s 179ms/step
inspirational
1/1 [==============================] - 0s 239ms/step
humor
1/1 [==============================] - 0s 320ms/step
deep meaning
1/1 [==============================] - 0s 354ms/step
humor
1/1 [==============================] - 1s 543ms/step
inspirational
1/1 [==============================] - 1s 584ms/step
inspirational
1/1 [==============================] - 1s 764ms/step
inspirational
1/1 [==============================] - 1s 614ms/step
humor
1/1 [==============================] - 0s 449ms/step
humor
1/1 [==============================] - 0s 421ms/step
humor
1/1 [==============================] - 0s 437ms/step
humor
1/1 [==============================] - 0s 335ms/step
humor
1/1 [==============================] - 0s 339ms/step
deep meaning
1/1 [==============================] - 0s 342ms/step
in

KeyboardInterrupt: ignored